# 如何创建动态（自构建）链

:::info 前置条件

本指南假设您已熟悉以下内容：
- [LangChain 表达式语言 (LCEL)](/docs/concepts/lcel)
- [如何将任何函数转换为可运行的](/docs/how_to/functions)

:::

有时我们希望在运行时根据链的输入构建链的部分内容（[路由](/docs/how_to/routing/) 是最常见的例子）。我们可以利用 RunnableLambda 的一个非常有用的特性来创建这样的动态链，即如果 RunnableLambda 返回一个 Runnable，那么该 Runnable 本身会被调用。让我们来看一个示例。

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="llm"
/>


In [4]:
# | echo: false

from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-sonnet-20240229")

In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnablePassthrough, chain

contextualize_instructions = """将最新的用户问题转换为独立的问题，基于聊天历史。不要回答问题，只返回问题本身（不需要描述性文本）。"""
contextualize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_instructions),
        ("placeholder", "{chat_history}"),
        ("human", "{question}"),
    ]
)
contextualize_question = contextualize_prompt | llm | StrOutputParser()

qa_instructions = (
    """根据以下上下文回答用户问题：\n\n{context}。"""
)
qa_prompt = ChatPromptTemplate.from_messages(
    [("system", qa_instructions), ("human", "{question}")]
)


@chain
def contextualize_if_needed(input_: dict) -> Runnable:
    if input_.get("chat_history"):
        # 注意：这里返回的是另一个 Runnable，而不是实际的输出。
        return contextualize_question
    else:
        return RunnablePassthrough() | itemgetter("question")


@chain
def fake_retriever(input_: dict) -> str:
    return "2024 年埃及的人口约为 1.11 亿"


full_chain = (
    RunnablePassthrough.assign(question=contextualize_if_needed).assign(
        context=fake_retriever
    )
    | qa_prompt
    | llm
    | StrOutputParser()
)

full_chain.invoke(
    {
        "question": "埃及怎么样",
        "chat_history": [
            ("human", "印尼的人口是多少"),
            ("ai", "大约 2.76 亿"),
        ],
    }
)

"According to the context provided, Egypt's population in 2024 is estimated to be about 111 million."

关键在于 `contextualize_if_needed` 返回另一个 Runnable，而不是实际的输出。当完整链被执行时，这个返回的 Runnable 本身会被运行。

从跟踪中可以看到，由于我们传入了 chat_history，我们在完整链中执行了 contextualize_question 链：https://smith.langchain.com/public/9e0ae34c-4082-4f3f-beed-34a2a2f4c991/r

注意，返回的 Runnable 的流式处理、批处理等功能都得到了保留：

In [ ]:
for chunk in contextualize_if_needed.stream(
    {
        "question": "埃及怎么样",
        "chat_history": [
            ("human", "印尼的人口是多少"),
            ("ai", "大约 2.76 亿"),
        ],
    }
):
    print(chunk)

What
 is
 the
 population
 of
 Egypt
?


In [ ]:
# 批处理示例
list(
    contextualize_if_needed.batch(
        [
            {
                "question": "埃及怎么样",
                "chat_history": [
                    ("human", "印尼的人口是多少"),
                    ("ai", "大约 2.76 亿"),
                ],
            },
            {"question": "埃及怎么样"},  # 没有聊天历史
        ]
    )
)